# Finding the best Open-Source Python API/library for financial data.
Data is the foundamental brick of any financial study and project so, before getting started with one, each developer should firstly choose the right tool that enables him to find good quality data.

This paper presents a systematic evaluation of various open-source financial APIs, aiming to provide clarity and guidance to developers and researchers seeking to integrate financial data into their Python projects.

Our study assesses each API across multiple metrics, including response time, error handling, handling of null values and data cost. By rigorously analyzing these factors, we offer an objective comparison to aid users in selecting the optimal API for their and our specific requirements.


## Code structure
api_functionspy: This folder contains a python script for each API we evaluated. Each script has three function with the same input/output values  in order to have stardard methods to compare APIs.
helpers.py: This small file contains some methods that can be useful to clean, pickle and load dataframe.
evalutators.py: This file contains all the functions used in main.py to evalute APIs.
main.py: The mail file contains the process and the calculations we used to compare the APIs as we will describe in README.
README.md: This file, providing an overview of the project and instructions.

Usage to replicate this script:

Install the required Python libraries listed in `requirements.txt`.

Execute the main.py `script`.

## Performance metrics
As we said before, we focused on different performance metrics for different usages.
If not clearly specified, we will consider only APIs that offer free access to their data.
This is the list of parameters and features we are going to discuss and evaluate.

<ul> 
    <li>Last update of a quote price</li>
    <li>Longest daily timeserie available</li>
    <li>Minimum time frequency available </li>
    <li>Nan values in a timeserie</li>
    <li>Deviation from the average values in a timeserie</li>
    <li>Time delay of the request </li>
    <li>Minimum time frequency available </li>

</ul>

In main.py we will create a simple dataframe containing in the rows all the api or libraries we will consider and in the columns all the parameters we previously mentioned in order to have a overview of the comparison and in order to be able to filter specific caratteristics.


In [93]:
import requests
from timeit import timeit
import sys
from api_alpha_vintage import * 
from api_tingo import *
from api_yfinance import *
from api_eodhd import *
from evaluators import *
from api_finhub import *
from api_fmp import *
from api_poligonio import *

from api_pandas_datareader import *

sys.path.insert(0, '/api_functions/')

fmp_key = "wpy0OAHOmFPLDyGbZzZ9IQuBlPMGf6v3"
aplha_vantage_key = "P0FAXOA7EI26OK9C"
tingo_key = "ef8c1728f6f1409157ab6a4f1266dd7af5df1ad5"
iex_key = "pk_5b65eb8e38e744f0b90e312a5c638d1b"
eodhd_key = "660734df8f7450.97003170"
fin_hub_key = "co3jcjpr01qj6vn80uogco3jcjpr01qj6vn80up0"
finage_key = "API_KEY01UJ76AC66CG8HPDTMJ75CW2GWR8FY0A"
poligon_key = "yrLQ1dm3ZAujvqGds3ZBQq4zaE7X0R9a"

api_list = pd.Series(["Financial Modelling Prep", "Alpha Vantage", "Quandl", "Tingo",
                      "YFinance", "IEX Cloud", "EODHD" ,"FinHub", "Finage", "Poligon.io"
                    ])

comparison_df_columns = pd.Series(["Free Calls","First day available","Min Frequency",
                                   "Nan values", "Error Rate", "Price", "Time delay of the request"])

comparison_df = pd.DataFrame(index = api_list, columns=comparison_df_columns)

### TIME DELAY
In this section we calculate the time delay of the request.
For simplicity, we are considering the request returning the last quote for a given stock.
The same processes could be done with different endpoints.

This evalutaions consists in the mean of the time response (calculate with the python time module and expressed in seconds) of each request.
For each api, we insert the calculated average time in the comparison_df we have previously defined.

In [94]:
ticker = "AAPL"
iterations = 1

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_yfinance, ticker)
comparison_df.at["YFinance","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_alpha_vantage, ticker)
comparison_df.at["Alpha Vantage","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_eodhd, ticker)
comparison_df.at["EODHD","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_finhub, ticker)
comparison_df.at["FinHub","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_fmp, ticker)
comparison_df.at["Financial Modelling Prep","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_tingo, ticker)
comparison_df.at["Tingo","Time delay of the request"] = sum / iterations

sum = 0
for i in range(iterations):
    sum += time_response(latest_price_poligonio, ticker)
comparison_df.at["Poligon.io","Time delay of the request"] = sum / iterations


### COMPARISON OF VALUES, DEVIATIONS AND ERROR RATE
The aim of this section is to evalute the quality of the data we obtain from different APIs.
The main focus is to calculate, in a given time and for each APIs, how much is the error in comparison to the real value.
The error simply consists in the deviation from the real value that, in our case, is the last price returned from the APIs of a given ticker.

Since there is no way to determine it, the real value is the average of all the values returned by the different APIs in the same moment.

In [95]:
ticker = "AAPL"

value_comparison_df_columns = pd.Series(["Price", "Deviation""Error Rate"])
value_comparison_df = pd.DataFrame(index = api_list, columns=value_comparison_df_columns)

value_comparison_df.at["YFinance", "Price"] = int(latest_price_yfinance(ticker)["currentPrice"] )
value_comparison_df.at["Financial Modelling Prep", "Price"] = latest_price_fmp(ticker)[0]["price"]
value_comparison_df.at["Alpha Vantage", "Price"] = latest_price_alpha_vantage(ticker)["Global Quote"]["05. price"]
value_comparison_df.at["Quandl", "Price"] = np.nan
value_comparison_df.at["Tingo", "Price"] = latest_price_tingo(ticker)[0]
value_comparison_df.at["IEX Cloud", "Price"] = np.nan
value_comparison_df.at["EODHD", "Price"] = latest_price_eodhd(ticker)["close"]
value_comparison_df.at["FinHub", "Price"] = latest_price_finhub(ticker)["c"]
value_comparison_df.at["Finage", "Price"] = np.nan
value_comparison_df.at["Poligon.io", "Price"] =latest_price_poligonio(ticker)["results"][0]["c"]

comparison_df["Price"] = value_comparison_df["Price"]

value_comparison_df["Price"] = value_comparison_df["Price"].astype(float)

value_comparison_df["Deviation"] = value_comparison_df["Price"].mean() - value_comparison_df["Price"]

value_comparison_df["Error Rate"] = abs (value_comparison_df["Deviation"] / value_comparison_df["Price"].mean()  * 100 )

comparison_df["Error Rate"] = value_comparison_df["Error Rate"]

KeyError: 'Global Quote'

### LONGEST TIME SERIES AVAILABLE
For each API that provides free time series data, we find the first historical quote available for a given stock.

In [ ]:
comparison_df.at["Alpha Vantage","First day available", ] = historical_price_alpha_vantage("KO").index[1]
comparison_df.at["EODHD","First day available", ] = str(historical_price_eodhd(ticker).index[-1])[0:10]
comparison_df.at["Financial Modelling Prep","First day available", ] = historical_price_fmp("AAPL")["historical"][-1]["date"]
comparison_df.at["Tingo","First day available", ] = str(historical_price_tingo("SPY").index[0])[0:10]
comparison_df.at["YFinance","First day available", ] = str(historical_price_yfinance("SPY").index[0])[0:10]

### MINIMUM TIME FREQUENCY
For the minimun time frequency available, we directly read each documentation.
In this case, we provide the minimum time frequency available for open-source APIs.
Minimum time frequency can vary when paying a premium subscription.

In [ ]:
comparison_df.at["Financial Modelling Prep", "Min Frequency"] =np.nan
comparison_df.at["Alpha Vantage", "Min Frequency"] = "1min"
comparison_df.at["Quandl", "Min Frequency"] = np.nan
comparison_df.at["Tingo", "Min Frequency"] = "1day"
comparison_df.at["YFinance", "Min Frequency"] = "1min"
comparison_df.at["IEX Cloud", "Min Frequency"] = np.nan
comparison_df.at["EODHD", "Min Frequency"] = "1day"
comparison_df.at["FinHub", "Min Frequency"] = np.nan
comparison_df.at["Finage", "Min Frequency"] = np.nan
comparison_df.at["Poligon.io", "Min Frequency"] = np.nan



### MAXIMUM NUMBER OF FREE CALLS
Some APIs provide a maximum number of free calls in a given time frame.

We directly looked at their documentation and we completed the dataframe as follows.

The data is expressed in "number of free calls per day"

In [ ]:
comparison_df.at["Financial Modelling Prep", "Free Calls"] = 250
comparison_df.at["Alpha Vantage", "Free Calls"] = 25
comparison_df.at["Quandl", "Free Calls"] = 0
comparison_df.at["Tingo", "Free Calls"] = 1000
comparison_df.at["YFinance", "Free Calls"] = "Unlimited"
comparison_df.at["IEX Cloud", "Free Calls"] = 0
comparison_df.at["EODHD", "Free Calls"] = 5040
comparison_df.at["FinHub", "Free Calls"] = 5040
comparison_df.at["Finage", "Free Calls"] = 0
comparison_df.at["Poligon.io", "Free Calls"] = 7200

In [ ]:
comparison_df

,Free Calls,First day available,Min Frequency,Nan values,Error Rate,Time delay of the request
Financial Modelling Prep,250,2019-04-16,NaN,NaN,0.044524,0.515059
Alpha Vantage,25,2024-04-11,1min,NaN,0.044524,0.270535
Quandl,0,NaN,NaN,NaN,NaN,NaN
Tingo,1000,1993-01-29,1day,NaN,0.044524,0.357708
YFinance,Unlimited,1993-01-29,1min,NaN,0.267142,0.122515
IEX Cloud,0,NaN,NaN,NaN,NaN,NaN
EODHD,5040,2024-04-12,1day,NaN,0.044524,1.228995
FinHub,5040,NaN,NaN,NaN,0.044524,0.364659
Finage,0,NaN,NaN,NaN,NaN,NaN
Poligon.io,7200,NaN,NaN,NaN,0.044524,0.369058


### RESULTS

And... That's it. 
After all this evalutaions, we have a general overview and comparison between the most diffused open-source financial api.
You can find in the footer section a list of all the resources and the documentations used in this study.

In [96]:
comparison_df

,Free Calls,First day available,Min Frequency,Nan values,Error Rate,Price,Time delay of the request
Financial Modelling Prep,NaN,NaN,NaN,NaN,NaN,NaN,1.162994
Alpha Vantage,NaN,NaN,NaN,NaN,NaN,NaN,0.15775
Quandl,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tingo,NaN,NaN,NaN,NaN,NaN,NaN,0.367324
YFinance,NaN,NaN,NaN,NaN,NaN,NaN,0.054087
IEX Cloud,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EODHD,NaN,NaN,NaN,NaN,NaN,NaN,1.389298
FinHub,NaN,NaN,NaN,NaN,NaN,NaN,0.363736
Finage,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Poligon.io,NaN,NaN,NaN,NaN,NaN,NaN,0.323801
